<center>
<button data-commandLinker-command="application:toggle-zen" href="#">Toggle fullscreen</i> <i class="fa-solid fa-expand"></button>
</center>

- To run the code, click on the first cell (gray box) and press <kbd>Shift</kbd>+<kbd>Enter</kbd> (or click the play button) to run each cell.
- Or, select `Run All Cells` from the `Run` menu.
- Feel free to experiment, but if you need to restore the original code, reload this browser page. Any changes you make will be lost when you reload!
- When you leave the page, you might get a "Changes you made may not be saved" message, which you can ignore.

In [ ]:
# Install Awkward Array in the browser
import piplite; await piplite.install("awkward-cpp"); await piplite.install("awkward==2.5.0")

# Import normal libraries
import numpy as np
import awkward as ak

## Versatile Arrays
Awkward Arrays are general tree-like data structures, like JSON, but contiguous in memory and operated upon with compiled, vectorized code like NumPy.

They look like NumPy arrays:

In [ ]:
ak.Array([1, 2, 3])

Like NumPy, they can have multiple dimensions:

In [ ]:
ak.Array([
    [1, 2, 3],
    [4, 5, 6]
])

These dimensions can have varying lengths; arrays can be [ragged](https://en.wikipedia.org/wiki/Jagged_array):

In [ ]:
ak.Array([
    [1, 2, 3],
    [4],
    [5, 6]
])

Each dimension can contain missing values:

In [ ]:
ak.Array([
    [1, 2, 3],
    [4],
    [5, 6, None]
])

Awkward Arrays can store _numbers_:

In [ ]:
ak.Array([
    [3, 141], 
    [59, 26, 535], 
    [8]
])

They can also work with _dates_:

In [ ]:
ak.Array(
    [
        [np.datetime64("1815-12-10"), np.datetime64("1969-07-16")],
        [np.datetime64("1564-04-26")],
    ]
)

They can even work with _strings_:

In [ ]:
ak.Array(
    [
        [
            "Benjamin List",
            "David MacMillan",
        ],
        [
            "Emmanuelle Charpentier",
            "Jennifer A. Doudna",
        ],
    ]
)

Awkward Arrays can have structure through _records_:

In [ ]:
ak.Array(
    [
        [
            {"name": "Benjamin List", "age": 53},
            {"name": "David MacMillan", "age": 53},
        ],
        [
            {"name": "Emmanuelle Charpentier", "age": 52},
            {"name": "Jennifer A. Doudna", "age": 57},
        ],
        [
            {"name": "Akira Yoshino", "age": 73},
            {"name": "M. Stanley Whittingham", "age": 79},
            {"name": "John B. Goodenough", "age": 98},
        ],
    ]
)

In fact, Awkward Arrays can represent many kinds of jagged data. They can possess complex structures that mix records, and primitive types.

In [ ]:
ak.Array(
    [
        [
            {
                "name": "Benjamin List",
                "age": 53,
                "institutions": [
                    "University of Cologne",
                    "Max Planck Institute for Coal Research",
                    "Hokkaido University",
                ],
            },
            {
                "name": "David MacMillan",
                "age": 53,
                "institutions": None,
            },
        ]
    ]
)

They can even contain unions!

In [ ]:
ak.Array(
    [
        [np.datetime64("1815-12-10"), "Cassini"],
        [np.datetime64("1564-04-26")],
    ]
)

## NumPy-like interface

Awkward Array _looks like_ NumPy. It behaves identically to NumPy for regular arrays

In [ ]:
x = ak.Array([
    [1, 2, 3],
    [4, 5, 6]
]);

In [ ]:
ak.sum(x, axis=-1)

providing a similar high-level API, and implementing the [ufunc](https://numpy.org/doc/stable/reference/ufuncs.html) mechanism:

In [ ]:
powers_of_two = ak.Array(
    [
        [1, 2, 4],
        [None, 8],
        [16],
    ]
);

In [ ]:
ak.sum(powers_of_two)

But generalises to the tricky kinds of data that NumPy struggles to work with. It can perform reductions through varying length lists:

![](example-reduction-sum.svg)

In [ ]:
ak.sum(powers_of_two, axis=0)

## Lightweight structures
Awkward makes it east to pull apart record structures:

In [ ]:
nobel_prize_winner = ak.Array(
    [
        [
            {"name": "Benjamin List", "age": 53},
            {"name": "David MacMillan", "age": 53},
        ],
        [
            {"name": "Emmanuelle Charpentier", "age": 52},
            {"name": "Jennifer A. Doudna", "age": 57},
        ],
        [
            {"name": "Akira Yoshino", "age": 73},
            {"name": "M. Stanley Whittingham", "age": 79},
            {"name": "John B. Goodenough", "age": 98},
        ],
    ]
);

In [ ]:
nobel_prize_winner.name

In [ ]:
nobel_prize_winner.age

These records are lightweight, and simple to compose:

In [ ]:
nobel_prize_winner_with_birth_year = ak.zip({
    "name": nobel_prize_winner.name,
    "age": nobel_prize_winner.age,
    "birth_year": 2021 - nobel_prize_winner.age
});

In [ ]:
nobel_prize_winner_with_birth_year.show()

## High performance
Like NumPy, Awkward Array performs computations in fast, optimised kernels.

In [ ]:
large_array = ak.Array([[1, 2, 3], [], [4, 5]] * 1_000_000)

We can compute the sum in `3.37 ms ± 107 µs` on a reference CPU:

In [ ]:
ak.sum(large_array)

The same sum can be computed with pure-Python over the flattened array in `369 ms ± 8.07 ms`:

In [ ]:
large_flat_array = ak.ravel(large_array)

sum(large_flat_array)

These performance values are not benchmarks; they are only an indication of the speed of Awkward Array.

Some problems are hard to solve with array-oriented programming. Awkward Array supports [Numba](https://numba.pydata.org/) out of the box:

```python
import numba as nb
ak.numba.register_and_check()

@nb.njit
def cumulative_sum(arr):
    result = 0
    for x in arr:
        for y in x:
            result += y
    return result
    
result = cumulative_sum(large_array)
```